In [1]:
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import scipy
import time

sys.path.append('./src')
import algorithms_approx3 as LcGD
from generative_graph_models import initialize_sparse_L, normalize_sparse_L
from algorithms_optimal import cvx_optimizer
from algorithms_approx3 import (objective_f, GD_optimizer_dir, 
                                opposite_view_heuristic_dir, popularity_heuristic_dir)

from utils import set_mkl_threads, assert_results
from generative_opinions_models import define_set_opinions, standardize
import importlib
set_mkl_threads(16)

importlib.reload(LcGD)

<module 'algorithms_approx3' from '/data/big/fcinus/FeedsRank/./src/algorithms_approx3.py'>

## Synthetic graph initialization

In [45]:
def make_doubly_stochastic(M, ε=1e-3):
    from sklearn.preprocessing import normalize
    i = 0
    while np.any(np.absolute(M.sum(axis=1)-np.ones(M.shape[0])) > ε):
        M = normalize(M, axis=1, norm='l1')
        M = normalize(M, axis=0, norm='l1')
        if i == 10000:
            print("reached max")
            break
        i += 1
    return M

In [86]:

n = 100
G = nx.erdos_renyi_graph(n, .05)
A = nx.adjacency_matrix(G)

I = np.identity(n)
z_eq = np.random.uniform(size=(n, 1))
A_eq = make_doubly_stochastic(A)
s = (I + I-A_eq) @ z_eq

e = A_eq.count_nonzero()
D_perc = e/(n*(n-1))*100
print(D_perc)

/tmp/ipykernel_1903691/256786180.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


5.151515151515151


In [87]:
A_opt, _, stats = cvx_optimizer(A_eq.todense(), s, verbosity=3)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) May 12 10:04:50 AM: Your problem has 10000 variables, 9945 constraints, and 0 parameters.
(CVXPY) May 12 10:04:51 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 12 10:04:51 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 12 10:04:51 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 12 10:04:51 AM: Compiling problem (target solver=SCS).
(CVXPY) May 12 10:04:51 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

In [88]:
def objective_f(z, A, sep=False):
    from scipy.sparse import identity
    I = identity(len(z))
    if sep: 
        return ((0.5*z.T @ (I + scipy.sparse.diags(np.array(A.sum(axis=0)).flatten()) - (A+A.T)) @ z + z.T @ z).item(),
                (z.T @ z).item())
    return (0.5*z.T @ (I + scipy.sparse.diags(np.array(A.sum(axis=0)).flatten()) - (A+A.T)) @ z + z.T @ z).item()

In [89]:
z, _ = scipy.sparse.linalg.bicg(2*I - A_opt, s, tol=1e-6)
z = z.reshape((len(s), 1))
objective_f(z, A_opt, False)

43.374498026151926

In [90]:
grad_params = {'budget': 1.}
lr_params = {'lr_routine': 'constant', 'lr_coeff': .1}
A_final, objectives, stats = LcGD.GD_optimizer_dir(A_eq, s, 100, True, 
                                                "ADAM", grad_params, lr_params, verbosity=1)

  Initializing LcGD ADAM solver with η=0.1
  t=1, obj=100.000, total_red=33.980, Δ_obj=0.00000, thr=0.000510
  t=2, obj=98.493, total_red=33.468, Δ_obj=0.00000, thr=0.000510
  t=3, obj=97.814, total_red=33.238, Δ_obj=0.00000, thr=0.000510
  t=4, obj=97.304, total_red=33.064, Δ_obj=0.00000, thr=0.000510
  t=5, obj=97.148, total_red=33.011, Δ_obj=0.00000, thr=0.000510
  t=6, obj=96.437, total_red=32.770, Δ_obj=0.00000, thr=0.000510
  t=7, obj=95.546, total_red=32.467, Δ_obj=0.00000, thr=0.000510
  t=8, obj=94.689, total_red=32.176, Δ_obj=0.00000, thr=0.000510
  t=9, obj=93.709, total_red=31.843, Δ_obj=0.00000, thr=0.000510
  t=10, obj=92.707, total_red=31.502, Δ_obj=0.00000, thr=0.000510
  t=11, obj=92.117, total_red=31.302, Δ_obj=0.91880, thr=0.000510
  t=12, obj=91.687, total_red=31.156, Δ_obj=0.73816, thr=0.000510
  t=13, obj=91.573, total_red=31.117, Δ_obj=0.47265, thr=0.000510
  t=14, obj=91.347, total_red=31.040, Δ_obj=0.25219, thr=0.000510
  t=15, obj=91.038, total_red=30.935, Δ_o

In [91]:
z, _ = scipy.sparse.linalg.bicg(2*I - A_final, s, tol=1e-6)
z = z.reshape((len(s), 1))
objective_f(z, A_opt, False)

37.593325739531124

## Real large graph definition

In [2]:
from sklearn.preprocessing import normalize

n = 40
A_eq = np.random.choice(3, size=(n, n))
np.fill_diagonal(A_eq, 0.)
A_eq = normalize(A_eq, axis=1, norm='l1')
A_eq = scipy.sparse.csr_matrix(A_eq)

I = np.identity(n)
z_eq = np.random.uniform(size=(n, 1))
s = (I + I-A_eq) @ z_eq
A_eq.count_nonzero()

1029

# Experiments

In [2]:
# graph
folder_path = Path("./datasets/experiments/directed/approximation_large_task__degree_constraint/")
simulation_folder = Path("7bf676cc5c5ae7f140e03ffe7b61671b")

s = np.load(folder_path / simulation_folder / Path("internal_opinions_sample00_python.npy"))
A_eq = scipy.sparse.load_npz(folder_path / simulation_folder / Path("A_eq_sparse_sample00_python.npz"))
z_eq = np.load(folder_path / simulation_folder / Path("eq_opinions_sample00_python.npy"))
#with open(folder_path / simulation_folder / Path("experiments_data_method.pkl"), "rb") as file:
#    exps = pickle.load(file)

I = scipy.sparse.identity(len(s))
print(A_eq.count_nonzero(), len(s))

31335568 2070819


In [3]:
importlib.reload(LcGD)
## Baseline 1
A_oppo, _, _ = opposite_view_heuristic_dir(A_eq, s, directed=True)
z, _ = scipy.sparse.linalg.bicg(2*I-A_oppo, s, tol=1e-6)
z = z.reshape((len(s), 1))
print(LcGD.objective_f(z, A_oppo) / LcGD.objective_f(z_eq, A_eq))
print(LcGD.objective_f(z, A_oppo, True), LcGD.objective_f(z_eq, A_eq, True))

# Baseline 2
A_pop, _, _ = popularity_heuristic_dir(A_eq, s, directed=True)
z, _ = scipy.sparse.linalg.bicg(2*I-A_pop, s, tol=1e-6)
z = z.reshape((len(s), 1))
print(LcGD.objective_f(z, A_pop) / LcGD.objective_f(z_eq, A_eq))
print(LcGD.objective_f(z, A_pop, True), LcGD.objective_f(z_eq, A_eq, True))


0.902535942791468
(805557.7471776066, 387079.7365911892) (892549.2149222158, 446708.6175766731)
1.004808883682259
(896841.3802774684, 457266.49588525575) (892549.2149222158, 446708.6175766731)


In [5]:
importlib.reload(LcGD)
grad_params = {'budget': 1.}
lr_params = {'lr_routine': 'constant', 'lr_coeff': .1}
A_final, objectives, stats = LcGD.GD_optimizer_dir(A_eq, s, 100, True, 
                                                    "ADAM", grad_params, lr_params, verbosity=1)

  Initializing LcGD ADAM solver with η=0.1
  t=1, obj=100.000, total_red=18.986, Δ_obj=0.00000, thr=31.335568
  t=2, obj=89.883, total_red=17.065, Δ_obj=0.00000, thr=31.335568
  t=3, obj=87.988, total_red=16.706, Δ_obj=0.00000, thr=31.335568
  t=4, obj=87.039, total_red=16.525, Δ_obj=0.00000, thr=31.335568
  t=5, obj=86.494, total_red=16.422, Δ_obj=0.00000, thr=31.335568
  t=6, obj=86.122, total_red=16.351, Δ_obj=0.00000, thr=31.335568
  t=7, obj=85.907, total_red=16.310, Δ_obj=0.00000, thr=31.335568
  t=8, obj=85.751, total_red=16.281, Δ_obj=0.00000, thr=31.335568
  t=9, obj=85.632, total_red=16.258, Δ_obj=0.00000, thr=31.335568
  t=10, obj=85.543, total_red=16.241, Δ_obj=0.00000, thr=31.335568
  t=11, obj=85.464, total_red=16.226, Δ_obj=1858.57025, thr=31.335568


KeyboardInterrupt: 

In [6]:
from generative_graph_models import assert_adj_stochasticity

assert_adj_stochasticity(A_final, True)


In [42]:
G.number_of_edges()/(G.number_of_nodes()*(G.number_of_nodes()-1)) * 100

0.012823901376369086